In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [20]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [7]:
db1 = DFDB('../trial/lgbm.pkl', auto_commit=False)

In [8]:
db2 = DFDB('../trial2/lgbm.pkl', auto_commit=False)

In [9]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [10]:
len(tsfresh_columns)

1071

In [11]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [12]:
df_trial1 = db1.select()
df_trial2 = db2.select()

In [17]:
df_trial1['kfold'] = df_trial1['param'].apply(lambda x : x['kfold'])
df_trial2['kfold'] = df_trial2['param'].apply(lambda x : x['kfold'])

In [57]:
df_trial1[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].head()

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-05 14:59:43.350233,1684,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",None,1.648266,0.000050,1.791306,0.000354,0.143040,None
1,2019-05-05 15:09:23.957708,948,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",None,1.669024,0.000039,1.802078,0.000546,0.133054,None
2,2019-05-05 15:25:03.474679,1684,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",None,1.648266,0.000050,1.791306,0.000354,0.143040,None
3,2019-05-05 15:34:17.148113,1664,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",None,1.647134,0.000056,1.793712,0.000402,0.146578,None
4,2019-05-05 15:42:32.778188,1644,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",None,1.643154,0.000037,1.794147,0.000326,0.150992,None


In [33]:
df_trial2[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].head()

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-17 03:01:27.907152,1071,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",group3 RFE,1.695777,0.000964,2.150657,0.007309,0.454880,group3 RFE
1,2019-05-17 03:04:52.040838,1051,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",group3 RFE,1.693611,0.001066,2.145472,0.006920,0.451861,group3 RFE
2,2019-05-17 03:08:18.681115,1031,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",group3 RFE,1.693420,0.000718,2.140937,0.006517,0.447517,group3 RFE
3,2019-05-17 03:11:15.799947,1011,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",group3 RFE,1.696996,0.000994,2.136962,0.006539,0.439965,group3 RFE
4,2019-05-17 03:14:07.687051,991,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",group3 RFE,1.704636,0.001043,2.128099,0.006891,0.423463,group3 RFE


In [55]:
n_splits_3_type_group_columns = df_trial2.loc[1]['param']['columns']
n_splits_8_type_stratified_columns = df_trial1.loc[114]['param']['columns']
n_splits_8_type_group_columns = df_trial1.loc[581]['param']['columns']
n_splits_8_type_timeseries_columns = df_trial1.loc[603]['param']['columns']

In [61]:
args

{'algorithm': {'cls': 'lgb.LGBMRegressor',
  'fit': {'early_stopping_rounds': 200,
   'eval_metric': 'mae',
   'verbose': False},
  'init': {'bagging_fraction': 0.9629636521622223,
   'feature_fraction': 0.917549020490175,
   'lambda_l1': 75.11011819901437,
   'lambda_l2': 84.85130517060821,
   'learning_rate': 0.1921537121698339,
   'max_bin': 36,
   'max_depth': 5,
   'min_data_in_leaf': 687,
   'n_jobs': 48,
   'random_state': 9950}},
 'columns': ['q25_roll_std_100'],
 'feature_importance': {'is_output': True,
  'permutation_feature_importance': True,
  'permutation_random_state': 1},
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}}}

In [67]:
mytrial = []
selected_columns = []
cv_score = 9999
for col in n_splits_8_type_group_columns:
    args = copy.deepcopy(df_trial1.loc[581]['param'])
    args['columns'] = selected_columns + [col]
    args['scaler']['init'] = {}
    args['algorithm']['fit'] = {}
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='revert RFE')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    if val_mae_mean < cv_score:
        selected_columns.append(col)
        cv_score = val_mae_mean

In [68]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-21 23:34:34.811476,1,revert RFE,2.062553,0.013677,2.114975,0.568713,0.052422,revert RFE
1,2019-05-21 23:34:47.928868,2,revert RFE,2.060227,0.013760,2.115777,0.568619,0.055550,revert RFE
2,2019-05-21 23:35:02.798591,2,revert RFE,2.059318,0.013812,2.113166,0.575136,0.053848,revert RFE
3,2019-05-21 23:35:22.144990,3,revert RFE,2.046225,0.013295,2.120690,0.571184,0.074466,revert RFE
4,2019-05-21 23:35:40.838223,3,revert RFE,2.052820,0.013640,2.119753,0.566104,0.066933,revert RFE
5,2019-05-21 23:36:00.930914,3,revert RFE,2.018824,0.012556,2.095364,0.524761,0.076541,revert RFE
6,2019-05-21 23:36:22.603310,4,revert RFE,2.010956,0.012539,2.097111,0.528261,0.086156,revert RFE
7,2019-05-21 23:36:44.148401,4,revert RFE,1.995110,0.011979,2.090560,0.514510,0.095449,revert RFE
8,2019-05-21 23:37:06.061323,5,revert RFE,1.967279,0.011177,2.080282,0.495529,0.113004,revert RFE
9,2019-05-21 23:37:27.805548,6,revert RFE,1.956264,0.011264,2.075991,0.494368,0.119727,revert RFE


In [16]:
#check feature_importances
# df_feature_importances = df_trial.loc[1398]['df_feature_importances']
# if type(df_feature_importances)==pd.DataFrame:
#     sorted_columns = EP.evaluate(df_feature_importances, key='average_permutation_weight')
# else:
#     sorted_columns = df_trial.loc[1398]['param']['columns']
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False)
# len(sorted_columns)

In [11]:
param = {'columns': tsfresh_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.19329183957823715,
   'feature_fraction': 0.7363782645537933,
   'bagging_fraction': 0.8585309819625903,
   'min_data_in_leaf': 351,
   'lambda_l1': 95.63411922439124,
   'lambda_l2': 48.949784919880365,
   'max_bin': 25,
   'num_leaves': 8,
   'random_state': 5040,
   'n_jobs': 32},
  'fit': {}}}

In [14]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial)

In [13]:
#  select features by permutation_weight
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=20, key='average_permutation_weight', remark='group3 RFE')

In [15]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-17 03:01:27.907152,1071,group3 RFE,1.695777,0.000964,2.150657,0.007309,0.454880,group3 RFE
1,2019-05-17 03:04:52.040838,1051,group3 RFE,1.693611,0.001066,2.145472,0.006920,0.451861,group3 RFE
2,2019-05-17 03:08:18.681115,1031,group3 RFE,1.693420,0.000718,2.140937,0.006517,0.447517,group3 RFE
3,2019-05-17 03:11:15.799947,1011,group3 RFE,1.696996,0.000994,2.136962,0.006539,0.439965,group3 RFE
4,2019-05-17 03:14:07.687051,991,group3 RFE,1.704636,0.001043,2.128099,0.006891,0.423463,group3 RFE
5,2019-05-17 03:16:55.988187,971,group3 RFE,1.713904,0.001170,2.124324,0.007848,0.410420,group3 RFE
6,2019-05-17 03:19:41.497957,951,group3 RFE,1.716507,0.001122,2.111514,0.008150,0.395007,group3 RFE
7,2019-05-17 03:22:22.556044,931,group3 RFE,1.717994,0.001174,2.102486,0.009762,0.384492,group3 RFE
8,2019-05-17 03:24:58.576704,911,group3 RFE,1.723458,0.001438,2.100919,0.008995,0.377461,group3 RFE
9,2019-05-17 03:27:30.541108,891,group3 RFE,1.723997,0.001454,2.097178,0.010448,0.373180,group3 RFE


In [18]:
#  tune hypterparameters
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200,800)
    lambda_l1 = trial.suggest_uniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_uniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 128)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':copy.deepcopy(df_trial.loc[51].param['columns']),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'lgb.LGBMRegressor',
            'init':{
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':32
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 51')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-17 04:28:31,885] Finished trial#0 resulted in value: 2.053388095789478. Current best value is 2.053388095789478 with parameters: {'learning_rate': 0.15421249597945286, 'feature_fraction': 0.9156010009640394, 'bagging_fraction': 0.703075175753202, 'min_data_in_leaf': 543, 'lambda_l1': 45.39931368612318, 'lambda_l2': 72.95267296100917, 'max_bin': 35, 'num_leaves': 16, 'random_state': 1639}.
[I 2019-05-17 04:28:33,080] Finished trial#1 resulted in value: 2.1184649668789746. Current best value is 2.053388095789478 with parameters: {'learning_rate': 0.15421249597945286, 'feature_fraction': 0.9156010009640394, 'bagging_fraction': 0.703075175753202, 'min_data_in_leaf': 543, 'lambda_l1': 45.39931368612318, 'lambda_l2': 72.95267296100917, 'max_bin': 35, 'num_leaves': 16, 'random_state': 1639}.
[I 2019-05-17 04:28:33,925] Finished trial#2 resulted in value: 2.211631467112031. Current best value is 2.053388095789478 with parameters: {'learning_rate': 0.15421249597945286, 'feature_fract

[I 2019-05-17 04:29:19,164] Finished trial#40 resulted in value: 2.041740003890214. Current best value is 2.041740003890214 with parameters: {'learning_rate': 0.09312516058358908, 'feature_fraction': 0.8760468436621816, 'bagging_fraction': 0.966297582901771, 'min_data_in_leaf': 637, 'lambda_l1': 99.9370007743052, 'lambda_l2': 17.258389619573347, 'max_bin': 94, 'num_leaves': 79, 'random_state': 9224}.
[I 2019-05-17 04:29:20,173] Finished trial#41 resulted in value: 2.041291306746265. Current best value is 2.041291306746265 with parameters: {'learning_rate': 0.09439210768569474, 'feature_fraction': 0.8812687072278574, 'bagging_fraction': 0.9754069129077927, 'min_data_in_leaf': 635, 'lambda_l1': 99.82544788152981, 'lambda_l2': 16.715532943340307, 'max_bin': 95, 'num_leaves': 21, 'random_state': 8916}.
[I 2019-05-17 04:29:21,143] Finished trial#42 resulted in value: 2.085562827480573. Current best value is 2.041291306746265 with parameters: {'learning_rate': 0.09439210768569474, 'feature_f

[I 2019-05-17 04:30:01,160] Finished trial#80 resulted in value: 2.0426353337586023. Current best value is 2.040798370573841 with parameters: {'learning_rate': 0.07738343290768636, 'feature_fraction': 0.842153422955133, 'bagging_fraction': 0.9389302212336406, 'min_data_in_leaf': 525, 'lambda_l1': 82.4262291590609, 'lambda_l2': 57.92514503014522, 'max_bin': 94, 'num_leaves': 26, 'random_state': 7844}.
[I 2019-05-17 04:30:02,098] Finished trial#81 resulted in value: 2.0520403522586697. Current best value is 2.040798370573841 with parameters: {'learning_rate': 0.07738343290768636, 'feature_fraction': 0.842153422955133, 'bagging_fraction': 0.9389302212336406, 'min_data_in_leaf': 525, 'lambda_l1': 82.4262291590609, 'lambda_l2': 57.92514503014522, 'max_bin': 94, 'num_leaves': 26, 'random_state': 7844}.
[I 2019-05-17 04:30:02,904] Finished trial#82 resulted in value: 2.079067685229095. Current best value is 2.040798370573841 with parameters: {'learning_rate': 0.07738343290768636, 'feature_fra

[I 2019-05-17 04:30:55,297] Finished trial#120 resulted in value: 2.0432837738456695. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feature_fraction': 0.8072630480461591, 'bagging_fraction': 0.638314058193034, 'min_data_in_leaf': 202, 'lambda_l1': 21.612542316147177, 'lambda_l2': 92.18633425943278, 'max_bin': 63, 'num_leaves': 46, 'random_state': 221}.
[I 2019-05-17 04:30:56,388] Finished trial#121 resulted in value: 2.058234983968999. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feature_fraction': 0.8072630480461591, 'bagging_fraction': 0.638314058193034, 'min_data_in_leaf': 202, 'lambda_l1': 21.612542316147177, 'lambda_l2': 92.18633425943278, 'max_bin': 63, 'num_leaves': 46, 'random_state': 221}.
[I 2019-05-17 04:30:57,551] Finished trial#122 resulted in value: 2.211229020749941. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feat

[I 2019-05-17 04:31:49,076] Finished trial#160 resulted in value: 2.232063877781938. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feature_fraction': 0.8072630480461591, 'bagging_fraction': 0.638314058193034, 'min_data_in_leaf': 202, 'lambda_l1': 21.612542316147177, 'lambda_l2': 92.18633425943278, 'max_bin': 63, 'num_leaves': 46, 'random_state': 221}.
[I 2019-05-17 04:31:50,165] Finished trial#161 resulted in value: 2.0445204769437324. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feature_fraction': 0.8072630480461591, 'bagging_fraction': 0.638314058193034, 'min_data_in_leaf': 202, 'lambda_l1': 21.612542316147177, 'lambda_l2': 92.18633425943278, 'max_bin': 63, 'num_leaves': 46, 'random_state': 221}.
[I 2019-05-17 04:31:51,824] Finished trial#162 resulted in value: 2.067320161245538. Current best value is 2.0317941901917824 with parameters: {'learning_rate': 0.05741051964332199, 'feat

In [22]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[(df_trial['remark']=='tune 51')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
114,2019-05-17 04:29:39.293976,51,group,1.961986,0.002968,2.061674,0.012845,0.099688,tune 51
65,2019-05-17 04:28:43.720094,51,group,2.001404,0.003266,2.069166,0.014335,0.067762,tune 51
136,2019-05-17 04:30:02.891071,51,group,1.995090,0.002538,2.079068,0.010742,0.083977,tune 51
227,2019-05-17 04:32:03.980555,51,group,2.039904,0.002782,2.093538,0.010386,0.053634,tune 51
151,2019-05-17 04:30:21.851321,51,group,2.026673,0.002208,2.113905,0.009500,0.087232,tune 51
247,2019-05-17 04:32:26.453876,51,group,2.031606,0.001781,2.122281,0.009160,0.090675,tune 51
61,2019-05-17 04:28:39.338793,51,group,2.088148,0.002094,2.164863,0.009292,0.076715,tune 51
133,2019-05-17 04:30:00.116849,51,group,2.140188,0.002825,2.170632,0.009804,0.030444,tune 51
147,2019-05-17 04:30:16.529659,51,group,2.135161,0.002215,2.201859,0.009620,0.066698,tune 51
176,2019-05-17 04:30:57.531793,51,group,2.141758,0.002183,2.211229,0.009289,0.069471,tune 51


In [24]:
param = {'algorithm': {'cls': 'lgb.LGBMRegressor',
  'fit': {},
  'init': {'bagging_fraction': 0.7744952101902622,
   'feature_fraction': 0.6070682774384075,
   'lambda_l1': 0.012232625023759935,
   'lambda_l2': 99.92837550563608,
   'learning_rate': 0.06504936893322744,
   'max_bin': 10,
   'min_data_in_leaf': 407,
   'n_jobs': 32,
   'num_leaves': 4,
   'random_state': 190}},
 'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'abs_q25_5',
  'spkt_welch_density__coeff_3',
  'spkt_welch_densitycoeff_2',
  "number_peaks{'n': 10}",
  'q05_roll_std_1000',
  'q05_roll_std_100',
  'abs_q75_7',
  'iqr_6',
  'abs_q75_6',
  '5000quantile99median_',
  '5000median_variance_',
  '3th_peak_freq',
  '5000smoothness_quantile05',
  "autocorrelation{'lag': 5}",
  'q05_5',
  'abs_q95_2',
  'kurt_1',
  'FFT_Mag_95q10000',
  'q05_roll_std_10',
  'abs_max_4',
  '5000skewness_max_',
  '5000smoothness_std_',
  'spkt_welch_density__coeff_4',
  'spkt_welch_density__coeff_42',
  "number_peaks{'n': 5}",
  'abs_q95_3',
  'fft_coefficientcoeff_6__attr_"abs"',
  'abs_max_8',
  'ar_coefficientk_10__coeff_3',
  'min_roll_std_100',
  '5000quantile99quantile75',
  'median__roll_std',
  '5000peak_to_average_power_ratio_rms_',
  'partial_autocorrelationlag_1',
  'ave10_7',
  'classic_sta_lta7_mean',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'q75_9',
  'fft_coefficientcoeff_24__attr_"angle"',
  '5000quantile01quantile25',
  'spkt_welch_density__coeff_64',
  'spkt_welch_density__coeff_9',
  'kurt_2',
  'abs_q99_2',
  '5000crest_factor_quantile99',
  'spkt_welch_density__coeff_38',
  'spkt_welch_density__coeff_65',
  'fft_coefficientcoeff_80__attr_"imag"',
  'fft_coefficientcoeff_19__attr_"abs"'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'}}

In [25]:
#  select features by permutation_weight
EP.select_features_(df_train, param, mytrial, nfeats_best=20, nfeats_removed_per_try=2, key='average_permutation_weight', remark='group3 RFE from 51feats')

In [26]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[df_trial['remark']=='group3 RFE from 51feats'][['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
254,2019-05-17 05:33:52.140704,51,group,2.000854,0.003298,2.065917,0.014304,0.065063
255,2019-05-17 05:33:57.015011,49,group,2.001237,0.003296,2.066253,0.013898,0.065016
256,2019-05-17 05:34:01.792692,47,group,2.001996,0.003341,2.066242,0.014080,0.064245
257,2019-05-17 05:34:06.409001,45,group,2.000923,0.003261,2.066377,0.014602,0.065454
258,2019-05-17 05:34:10.910126,43,group,2.000845,0.003291,2.067335,0.014293,0.066490
259,2019-05-17 05:34:15.010734,41,group,2.000669,0.003252,2.065495,0.014319,0.064826
260,2019-05-17 05:34:19.143286,39,group,2.000791,0.003178,2.066922,0.014336,0.066131
261,2019-05-17 05:34:22.921449,37,group,2.001215,0.003294,2.066767,0.014066,0.065552
262,2019-05-17 05:34:26.610112,35,group,2.000846,0.003292,2.066260,0.014426,0.065414
263,2019-05-17 05:34:30.115418,33,group,2.003375,0.003382,2.066243,0.014502,0.062868


In [30]:
#  tune hypterparameters
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200,800)
    lambda_l1 = trial.suggest_uniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_uniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 128)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':copy.deepcopy(df_trial.loc[269].param['columns']),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'lgb.LGBMRegressor',
            'init':{
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':32
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 269')
    
    return np.mean(df_his.valid)

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-17 06:08:40,646] Finished trial#0 resulted in value: 2.070997339498051. Current best value is 2.070997339498051 with parameters: {'learning_rate': 0.14712530436880308, 'feature_fraction': 0.9528202832737862, 'bagging_fraction': 0.7884212845700415, 'min_data_in_leaf': 560, 'lambda_l1': 44.862209390360164, 'lambda_l2': 56.20909711481201, 'max_bin': 18, 'num_leaves': 59, 'random_state': 4697}.
[I 2019-05-17 06:08:41,389] Finished trial#1 resulted in value: 2.0747099351797926. Current best value is 2.070997339498051 with parameters: {'learning_rate': 0.14712530436880308, 'feature_fraction': 0.9528202832737862, 'bagging_fraction': 0.7884212845700415, 'min_data_in_leaf': 560, 'lambda_l1': 44.862209390360164, 'lambda_l2': 56.20909711481201, 'max_bin': 18, 'num_leaves': 59, 'random_state': 4697}.
[I 2019-05-17 06:08:42,117] Finished trial#2 resulted in value: 2.083405237224194. Current best value is 2.070997339498051 with parameters: {'learning_rate': 0.14712530436880308, 'feature_f

[I 2019-05-17 06:09:16,061] Finished trial#40 resulted in value: 2.0899999723226172. Current best value is 2.048949032214546 with parameters: {'learning_rate': 0.05647670571856972, 'feature_fraction': 0.6012600177322884, 'bagging_fraction': 0.832119074081954, 'min_data_in_leaf': 250, 'lambda_l1': 67.99509913034593, 'lambda_l2': 32.15077014706212, 'max_bin': 90, 'num_leaves': 75, 'random_state': 5707}.
[I 2019-05-17 06:09:16,717] Finished trial#41 resulted in value: 2.063555737860473. Current best value is 2.048949032214546 with parameters: {'learning_rate': 0.05647670571856972, 'feature_fraction': 0.6012600177322884, 'bagging_fraction': 0.832119074081954, 'min_data_in_leaf': 250, 'lambda_l1': 67.99509913034593, 'lambda_l2': 32.15077014706212, 'max_bin': 90, 'num_leaves': 75, 'random_state': 5707}.
[I 2019-05-17 06:09:17,644] Finished trial#42 resulted in value: 2.0796906479592514. Current best value is 2.048949032214546 with parameters: {'learning_rate': 0.05647670571856972, 'feature_f

[I 2019-05-17 06:09:52,783] Finished trial#80 resulted in value: 2.050840269309448. Current best value is 2.045269625622746 with parameters: {'learning_rate': 0.05107300670335299, 'feature_fraction': 0.613676210102545, 'bagging_fraction': 0.8479098806777672, 'min_data_in_leaf': 220, 'lambda_l1': 55.2531777029512, 'lambda_l2': 89.07109013977826, 'max_bin': 54, 'num_leaves': 49, 'random_state': 7212}.
[I 2019-05-17 06:09:54,109] Finished trial#81 resulted in value: 2.0992427511342115. Current best value is 2.045269625622746 with parameters: {'learning_rate': 0.05107300670335299, 'feature_fraction': 0.613676210102545, 'bagging_fraction': 0.8479098806777672, 'min_data_in_leaf': 220, 'lambda_l1': 55.2531777029512, 'lambda_l2': 89.07109013977826, 'max_bin': 54, 'num_leaves': 49, 'random_state': 7212}.
[I 2019-05-17 06:09:54,573] Finished trial#82 resulted in value: 2.0704265168980354. Current best value is 2.045269625622746 with parameters: {'learning_rate': 0.05107300670335299, 'feature_fra

[I 2019-05-17 06:10:32,433] Finished trial#120 resulted in value: 2.0673140549866305. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_fraction': 0.7352982428059609, 'bagging_fraction': 0.98862692884486, 'min_data_in_leaf': 200, 'lambda_l1': 32.11041086861768, 'lambda_l2': 60.788375782373336, 'max_bin': 86, 'num_leaves': 59, 'random_state': 3068}.
[I 2019-05-17 06:10:33,169] Finished trial#121 resulted in value: 2.2337965933355073. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_fraction': 0.7352982428059609, 'bagging_fraction': 0.98862692884486, 'min_data_in_leaf': 200, 'lambda_l1': 32.11041086861768, 'lambda_l2': 60.788375782373336, 'max_bin': 86, 'num_leaves': 59, 'random_state': 3068}.
[I 2019-05-17 06:10:33,971] Finished trial#122 resulted in value: 2.0888723750321763. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_

[I 2019-05-17 06:11:16,786] Finished trial#160 resulted in value: 2.0498276198654817. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_fraction': 0.7352982428059609, 'bagging_fraction': 0.98862692884486, 'min_data_in_leaf': 200, 'lambda_l1': 32.11041086861768, 'lambda_l2': 60.788375782373336, 'max_bin': 86, 'num_leaves': 59, 'random_state': 3068}.
[I 2019-05-17 06:11:17,470] Finished trial#161 resulted in value: 2.0548208798025835. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_fraction': 0.7352982428059609, 'bagging_fraction': 0.98862692884486, 'min_data_in_leaf': 200, 'lambda_l1': 32.11041086861768, 'lambda_l2': 60.788375782373336, 'max_bin': 86, 'num_leaves': 59, 'random_state': 3068}.
[I 2019-05-17 06:11:18,483] Finished trial#162 resulted in value: 2.0665323519880157. Current best value is 2.044054675944984 with parameters: {'learning_rate': 0.0431998413129995, 'feature_

In [39]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[(df_trial['remark']=='tune 269')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
391,2019-05-17 06:10:32.416213,21,group,1.979505,0.002285,2.067314,0.010692,0.087809,tune 269
348,2019-05-17 06:09:50.083822,21,group,1.999831,0.002132,2.090656,0.010334,0.090825,tune 269
376,2019-05-17 06:10:17.157425,21,group,2.011211,0.002560,2.097190,0.010735,0.085979,tune 269
279,2019-05-17 06:08:47.053815,21,group,2.018428,0.002238,2.102040,0.010247,0.083612,tune 269
363,2019-05-17 06:10:04.194142,21,group,2.055656,0.002466,2.113545,0.009309,0.057889,tune 269
408,2019-05-17 06:10:50.788932,21,group,2.064891,0.002278,2.132515,0.009524,0.067624,tune 269
341,2019-05-17 06:09:42.881627,21,group,2.104724,0.002320,2.164600,0.009322,0.059876,tune 269
454,2019-05-17 06:11:39.614360,21,group,2.144058,0.002401,2.193236,0.009909,0.049178,tune 269
421,2019-05-17 06:11:05.615279,21,group,2.151409,0.002456,2.202709,0.009596,0.051300,tune 269
288,2019-05-17 06:08:54.506953,21,group,2.153972,0.002490,2.214349,0.010081,0.060377,tune 269


In [50]:
param = copy.deepcopy(df_trial.loc[391]['param'])
param['kfold']['type'] = 'stratified'
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 391 use stratified')

In [51]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].tail(1)

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
671,2019-05-17 06:37:52.509487,21,stratified,1.997276,0.00001,2.017029,0.000066,0.019752,remodel 391 use stratified


In [40]:
#  tune hypterparameters
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200,800)
    lambda_l1 = trial.suggest_uniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_uniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 128)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':copy.deepcopy(df_trial.loc[269].param['columns']),
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'lgb.LGBMRegressor',
            'init':{
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'random_state':random_state,
                'n_jobs':32
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 269 by stratified')
    
    return np.mean(df_his.valid)

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-17 06:24:13,943] Finished trial#0 resulted in value: 1.8792640256932363. Current best value is 1.8792640256932363 with parameters: {'learning_rate': 0.24026703349670075, 'feature_fraction': 0.791481290239838, 'bagging_fraction': 0.7713028697595078, 'min_data_in_leaf': 529, 'lambda_l1': 12.547866725413856, 'lambda_l2': 55.08621451340376, 'max_bin': 66, 'num_leaves': 95, 'random_state': 2302}.
[I 2019-05-17 06:24:14,479] Finished trial#1 resulted in value: 1.9555485744640515. Current best value is 1.8792640256932363 with parameters: {'learning_rate': 0.24026703349670075, 'feature_fraction': 0.791481290239838, 'bagging_fraction': 0.7713028697595078, 'min_data_in_leaf': 529, 'lambda_l1': 12.547866725413856, 'lambda_l2': 55.08621451340376, 'max_bin': 66, 'num_leaves': 95, 'random_state': 2302}.
[I 2019-05-17 06:24:15,232] Finished trial#2 resulted in value: 1.9307051235635646. Current best value is 1.8792640256932363 with parameters: {'learning_rate': 0.24026703349670075, 'featur

[I 2019-05-17 06:24:52,202] Finished trial#40 resulted in value: 1.869619775339177. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:24:53,201] Finished trial#41 resulted in value: 1.8633514653812526. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:24:53,971] Finished trial#42 resulted in value: 1.9123376405933115. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_

[I 2019-05-17 06:25:40,113] Finished trial#80 resulted in value: 1.8219284502828934. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:25:41,603] Finished trial#81 resulted in value: 1.8155798525482876. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:25:43,091] Finished trial#82 resulted in value: 1.823869788412505. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_

[I 2019-05-17 06:26:25,991] Finished trial#120 resulted in value: 1.8774501139720414. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:26:27,184] Finished trial#121 resulted in value: 1.864137170054856. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'feature_fraction': 0.7395707847280799, 'bagging_fraction': 0.975564616422149, 'min_data_in_leaf': 200, 'lambda_l1': 15.529857560492736, 'lambda_l2': 44.86601380163444, 'max_bin': 99, 'num_leaves': 106, 'random_state': 3992}.
[I 2019-05-17 06:26:28,264] Finished trial#122 resulted in value: 1.8496764653545885. Current best value is 1.796182569773525 with parameters: {'learning_rate': 0.4903804965418004, 'featu

[I 2019-05-17 06:27:21,021] Finished trial#160 resulted in value: 1.827203536621152. Current best value is 1.7844820814446625 with parameters: {'learning_rate': 0.31942918677649834, 'feature_fraction': 0.9856819936906076, 'bagging_fraction': 0.6193447142397187, 'min_data_in_leaf': 200, 'lambda_l1': 7.9740522154945594, 'lambda_l2': 31.14570234103974, 'max_bin': 78, 'num_leaves': 104, 'random_state': 5366}.
[I 2019-05-17 06:27:22,549] Finished trial#161 resulted in value: 1.8238581452788427. Current best value is 1.7844820814446625 with parameters: {'learning_rate': 0.31942918677649834, 'feature_fraction': 0.9856819936906076, 'bagging_fraction': 0.6193447142397187, 'min_data_in_leaf': 200, 'lambda_l1': 7.9740522154945594, 'lambda_l2': 31.14570234103974, 'max_bin': 78, 'num_leaves': 104, 'random_state': 5366}.
[I 2019-05-17 06:27:24,350] Finished trial#162 resulted in value: 1.8239891099508372. Current best value is 1.7844820814446625 with parameters: {'learning_rate': 0.31942918677649834

[I 2019-05-17 06:28:12,212] Finished trial#198 resulted in value: 1.8799124095921702. Current best value is 1.7844820814446625 with parameters: {'learning_rate': 0.31942918677649834, 'feature_fraction': 0.9856819936906076, 'bagging_fraction': 0.6193447142397187, 'min_data_in_leaf': 200, 'lambda_l1': 7.9740522154945594, 'lambda_l2': 31.14570234103974, 'max_bin': 78, 'num_leaves': 104, 'random_state': 5366}.
[I 2019-05-17 06:28:13,124] Finished trial#199 resulted in value: 1.8992174925423804. Current best value is 1.7844820814446625 with parameters: {'learning_rate': 0.31942918677649834, 'feature_fraction': 0.9856819936906076, 'bagging_fraction': 0.6193447142397187, 'min_data_in_leaf': 200, 'lambda_l1': 7.9740522154945594, 'lambda_l2': 31.14570234103974, 'max_bin': 78, 'num_leaves': 104, 'random_state': 5366}.


In [45]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[(df_trial['remark']=='tune 269 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
477,2019-05-17 06:24:17.929310,21,stratified,1.907973,0.000022,1.950075,0.000090,0.042103,tune 269 by stratified
472,2019-05-17 06:24:14.477915,21,stratified,1.918838,0.000027,1.955549,0.000097,0.036711,tune 269 by stratified
564,2019-05-17 06:25:54.864944,21,stratified,1.944479,0.000025,1.983633,0.000036,0.039153,tune 269 by stratified
476,2019-05-17 06:24:17.476519,21,stratified,2.001504,0.000012,2.014615,0.000046,0.013110,tune 269 by stratified
486,2019-05-17 06:24:26.613097,21,stratified,2.065248,0.000010,2.087396,0.000049,0.022148,tune 269 by stratified


In [58]:
param = copy.deepcopy(df_trial.loc[477]['param'])
param['kfold']['type'] = 'group'
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 477 use group')

In [61]:
df_trial = pd.DataFrame(mytrial)
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[['datetime','nfeatures', 'kfold-type', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].tail(1)

,datetime,nfeatures,kfold-type,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
673,2019-05-17 06:50:10.345009,21,group,1.87808,0.002409,2.053583,0.014521,0.175503,remodel 477 use group


In [ ]:
# 477 tune 269  by stratified => 673
#  391 tune 269 by group remodel 391=> 671

In [62]:
df_trial.to_pickle('../trial/lgbm.pkl')

In [21]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_maylgbm_2024.csv', index=False)

In [23]:
import random
from deap import creator, base, tools, algorithms
import warnings
warnings.filterwarnings("default")
#https://deap.readthedocs.io/en/master/api/tools.html

class GAFeatureSelection(object):
    

    def __init__(self, df, features, n_component, n_population=None, tournsize=None, cxpb=.5, mutpb=.2, indpb=.05, ngen=100, rndseed=1985):
        random.seed(rndseed)
        self.df = df
        self.features = features
        self.n_component = n_component
        self.n_population = n_population
        if (self.n_population == None):
            self.n_population = self.__evalute_n_population__(self.n_component, len(self.features))
        self.tournsize = tournsize
        if (self.tournsize == None):
            self.tournsize = int(self.n_population*.01)
        self.cxpb = cxpb
        self.mutpb = mutpb
        self.indpb = indpb
        self.ngen = ngen
#         creator.create("FitnessMax", base.Fitness, weights=(1.0,),)
#         creator.create("Individual", list, fitness=creator.FitnessMax)

        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_gen", random.randint, 0, len(self.features)-1)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_gen, n=self.n_component)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", self.evaluate)
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutFlipBit, indpb=self.indpb)
        self.toolbox.register("select", tools.selTournament, tournsize=self.tournsize)
        self.population = self.toolbox.population(n=self.n_population)
        return
    
    
    def evaluate(self, individual):
        raise Exception('no inheritance')
        
        
    def run(self, ntop=1):
        for gen in range(self.ngen):
            offspring = algorithms.varAnd(self.population, self.toolbox, cxpb=self.cxpb, mutpb=self.mutpb)
            fits = self.toolbox.map(self.toolbox.evaluate, offspring)
            for fit, ind in zip(fits, offspring):
                ind.fitness.values = fit
            self.population = self.toolbox.select(offspring, k=len(self.population))
        return tools.selBest(self.population, k=ntop)
    
    
    def __evalute_n_population__(self, n_component, n_features, init_n_population=100, feature_adoption_rate=.9):
        n_population = init_n_population
        selected_n_feature = 0
        while selected_n_feature/n_features < feature_adoption_rate:
            n_population = int(n_population*1.2)
            k = []
            for j in np.arange(n_population):
                k.append([random.randint(0, n_features-1) for i in np.arange(n_component)])
            k = np.array(k)
            selected_n_feature = np.unique(k).shape[0]
        return n_population
    

In [39]:
ga_trial = []
fixed_columns = ['abs_q01_4',
             'spkt_welch_density__coeff_3',
             'abs_q75_7',
             'q05_roll_std_1000',
             'q25_roll_std_100',
             "number_peaks{'n': 10}",
             'spkt_welch_densitycoeff_2',
             'abs_max_7']

class MyGAFS(GAFeatureSelection):
    
#     def __init__(self, df, features, n_component, **kwargs):
#         super(MyGAFS, self).__init__(df, features, n_component, **kwargs)
#         return
    
    def evaluate(self, individual):
        
        select_i = [self.features[chromosome] for chromosome in individual]
        columns = fixed_columns + select_i
        
        param={
            'algorithm': {
                'cls': 'lgb.LGBMRegressor',
                'fit': {
                    'early_stopping_rounds': 200,
                    'eval_metric': 'mae',
                    'verbose': False
                },
                'init': {
                    'bagging_fraction': 0.9629636521622223,
                    'feature_fraction': 0.917549020490175,
                    'lambda_l1': 75.11011819901437,
                    'lambda_l2': 84.85130517060821,
                    'learning_rate': 0.1921537121698339,
                    'max_bin': 36,
                    'max_depth': 5,
                    'min_data_in_leaf': 687,
                    'n_jobs': 16,
                    'random_state': 9950
                }
                },
            'columns': columns,
            'feature_importance': {
                'is_output': False,
                'permutation_feature_importance': False,
                'permutation_random_state': 1
            },
            'kfold': {
                'n_splits': 8,
                'random_state': 1985,
                'shuffle': True,
                'type': 'group'
            },
            'scaler': {
                'cls': 'StandardScaler'
            }
        }
        
        score = []
        try:
            df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = None, trial=ga_trial, remark=None)
            val_mae_mean = np.mean(df_his.valid)
            val_mae_var = np.var(df_his.valid)
            train_mae_mean = np.mean(df_his.train)
            train_mae_var = np.var(df_his.train)
        except Exception as e:
            print(e.__str__())
            return 0
        fitness = np.abs(val_mae_mean-train_mae_mean)*val_mae_mean
        return fitness,

In [24]:
# gafs = MyGAFS(df = df_train,  features = df_train[tsfresh_columns].columns.drop(fixed_columns).tolist(),  n_component = 17,  ngen = 10,  n_population = 10000)
# top10 = gafs.run(10)